In [2]:
import pandas as pd
import pickle

from recommendation_model import RecommendationSystem

In [3]:
data = pd.read_csv('./data/Final_data.csv')
data.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,...,cal_from_macros,pct_carbs,protein_per_kg,pct_HRR,pct_maxHR,cal_balance,lean_mass_kg,expected_burn,Burns Calories (per 30 min)_bc,Burns_Calories_Bin
0,34.91,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,...,2139.59,0.500432,1.624789,0.741237,0.835985,725.10,47.777394,685.1600,7.260425e+19,Medium
1,23.37,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,...,1711.65,0.500850,1.514093,0.551247,0.734270,-232.91,40.809803,978.6184,1.020506e+20,High
2,33.20,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,...,1965.92,0.500610,1.663445,0.574534,0.708124,805.74,44.635580,654.5266,1.079607e+20,High
3,38.69,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,...,1627.28,0.499533,0.862017,0.744155,0.811150,1206.21,63.007432,773.6300,8.987921e+19,High
4,45.09,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,...,2659.23,0.500581,2.538153,0.668405,0.789751,303.60,43.347504,711.4176,5.264685e+19,Low


In [4]:
cleaned_data = data[[
    'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Experience_Level', 
    'Body Part', 'Name of Exercise', 'Difficulty Level', 'Target Muscle Group',
    'Benefit', 'Reps', 'Sets'
]]

In [5]:
def normalize_exercise_name(name):
    name = name.lower()
    if "push" in name:
        return "Push Ups"
    elif "squat" in name:
        return "Squats"
    elif "crunch" in name:
        return "Crunches"
    elif "plank" in name:
        return "Plank"
    elif "deadlift" in name:
        return "Deadlift"
    elif "curl" in name:
        return "Bicep Curl"
    elif "press" in name:
        return "Press"
    else:
        return name.title()
    
muscle_map = {
    "Push Ups": "Chest, Triceps, Shoulders",
    "Squats": "Legs, Glutes",
    "Crunches": "Abs, Core",
    "Plank": "Core",
    "Deadlift": "Back, Legs, Core",
    "Bicep Curl": "Biceps",
    "Press": "Chest, Shoulders, Triceps"
}

cleaned_data['Exercise_Base'] = cleaned_data['Name of Exercise'].apply(normalize_exercise_name)

cleaned_data['Target Muscle Group_Cleaned'] = cleaned_data['Exercise_Base'].map(muscle_map)
cleaned_data['Target Muscle Group_Cleaned'].fillna(cleaned_data['Target Muscle Group'], inplace = True)

cleaned_data = cleaned_data.drop(columns=['Name of Exercise', 'Target Muscle Group'])

cleaned_data.head()


C:\Users\retro\AppData\Local\Temp\ipykernel_15820\3276347806.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['Exercise_Base'] = cleaned_data['Name of Exercise'].apply(normalize_exercise_name)
C:\Users\retro\AppData\Local\Temp\ipykernel_15820\3276347806.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['Target Muscle Group_Cleaned'] = cleaned_data['Exercise_Base'].map(muscle_map)
C:\Users\retro\AppData\Local\Temp\ipykernel_15820\3276347806.py:33: FutureWarning: A value is tr

,Age,Gender,Weight (kg),Height (m),Experience_Level,Body Part,Difficulty Level,Benefit,Reps,Sets,Exercise_Base,Target Muscle Group_Cleaned
0,34.91,Male,65.27,1.62,2.01,Legs,Advanced,Improves shoulder health and posture,20.91,4.99,Push Ups,"Chest, Triceps, Shoulders"
1,23.37,Female,56.41,1.55,2.01,Chest,Intermediate,Strengthens lower abs,16.15,4.01,Bear Crawls,"Back, Core, Shoulders"
2,33.20,Female,58.98,1.67,1.02,Arms,Intermediate,Builds chest strength,21.90,5.00,Dips,"Quadriceps, Glutes"
3,38.69,Female,93.78,1.70,1.99,Shoulders,Advanced,Improves coordination and cardiovascular health,16.92,4.01,Mountain Climbers,"Biceps, Forearms"
4,45.09,Male,52.42,1.88,2.00,Abs,Advanced,Targets obliques and improves core rotation,15.01,4.99,Bicep Curl,Biceps


In [12]:
recsys = RecommendationSystem(cleaned_data)
print('\n'.join(recsys.get_recommendation('male', 1.0, 'core', 'Advanced', 5)))

dict_items([('Thrusters', 1), ('Plank', 1), ('Bear Crawls', 1), ('Dead Bugs', 1), ('Mountain Climbers', 1)])
Thrusters
Plank
Bear Crawls
Dead Bugs
Mountain Climbers


In [10]:
with open('./data/fitness_recommender.pkl', 'wb') as f:
    pickle.dump(recsys, f)